<a href="https://colab.research.google.com/github/TruscaPetre/AAIT-Nosy-Missing-Labels/blob/main/tutorial%20colab%20missing%20labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Missing Labels task 1

## Imports

In [6]:
import urllib
import shutil
import os
import time
import copy

import pandas as pd
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import random

from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import SubsetRandomSampler, Dataset
from torchvision import transforms, datasets 

from sklearn.metrics import classification_report

import itertools

In [2]:
task1_id = "1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd"

In [3]:
# replace here your ide &id=1iNTm3-9GSN2QjjITBi73ta73gDF_oacf"
# replace here your id 'https://docs.google.com/uc?export=download&id=1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd'
# replace here your target name -O task1.tar.gz &&
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd" -O task1.tar.gz && rm -rf /tmp/cookies.txt


--2022-12-28 21:26:33--  https://docs.google.com/uc?export=download&confirm=t&id=1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd
Resolving docs.google.com (docs.google.com)... 172.253.117.138, 172.253.117.102, 172.253.117.139, ...
Connecting to docs.google.com (docs.google.com)|172.253.117.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9nn2g07ta774n2dlgsnfpoku87248svd/1672262775000/08997952672865575084/*/1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd?e=download&uuid=48453833-cede-43e7-b63b-ce002c448029 [following]
--2022-12-28 21:26:33--  https://doc-0g-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9nn2g07ta774n2dlgsnfpoku87248svd/1672262775000/08997952672865575084/*/1dO1vqCoJm2xwrnr171A6_eW7ikd-alrd?e=download&uuid=48453833-cede-43e7-b63b-ce002c448029
Resolving doc-0g-8k-docs.googleusercontent.com (doc-0g-8k-docs.googleusercontent.com)... 173.194.203.

In [4]:
%%capture
!mkdir data
!mv task1.tar.gz ./data
!tar -xzvf "/content/data/task1.tar.gz" -C "/content/data/"     #[run this cell to extract tar.gz files]
# this may take 12 seconds

## Training on labeled data

### Download model

In [7]:
# Download a pytorch MobileNet pretrained model
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
# change the Linear output to fit our dataset

# the model has initially 1000 outputs
# print(model.classifier)
# > Sequential(
#   (0): Dropout(p=0.2)
#   (1): Linear(in_features=1280, out_features=1000, bias=True)
# )

model.classifier[1] = nn.Linear(1280, 100)
print(model.classifier)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=100, bias=True)
)


### Prepare datasets

In [8]:
# Organize data for pytorch training ( only once )

dir_data = 'data/task1/train_data/'
# Read the annotations file into a DataFrame
df = pd.read_csv(f'{dir_data}annotations.csv')

# Define the base directory
base_dir = 'data/task1/labeled'

# Iterate over the rows in the DataFrame
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Extract the path and class from the row
    path = row['sample']
    label = row['label']
    
    # Create the directory for the class
    class_dir = f'{base_dir}/{label}'
    os.makedirs(class_dir, exist_ok=True)
    
    # Copy the file to the class directory
    shutil.copy(f"data/{path}", class_dir) 

#TODO: here I don't want to split the datasets into training and testing
# because I want to use all the training and to create a new dataset
# which will create labels for the untrained samples


100%|██████████| 23555/23555 [00:06<00:00, 3890.09it/s]


In [10]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = 'data/task1/labeled'
image_dataset = datasets.ImageFolder(data_dir, preprocess) 
dataloader  = torch.utils.data.DataLoader(image_dataset, batch_size = 32, shuffle = True, num_workers = 4)
class_names = image_dataset.classes
dataset_size = len(image_dataset)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Start training

In [14]:
def train_loop(model, scheduler, optimizer, criterion, dataset_size, dataloader):
                     
    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
    
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    scheduler.step()

    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects.double() / dataset_size
    return model, epoch_loss, epoch_acc

def train_model(model, *args, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        model.train()  # Set model to training mode
        model, epoch_loss, epoch_acc = train_loop(model, *args)
        print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # deep copy the model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [12]:
# Setting hyperparameters

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model = train_model(model, 
                    exp_lr_scheduler, 
                    optimizer, 
                    criterion,  
                    dataset_size, 
                    dataloader, 
                    num_epochs=25)

In [ ]:
# TODO: save the model after each iteration
# TODO: create a function that creates the a new dataset for the second iteration
# TODO: restart the training on the new dataset
# TODO: put all the previous 3 steps in a loop 

## missing labels Theory


- The classification problem can be tackled using semi-supervised learning methods. There are 3 propular approaches to address semi-supervised learning problems:
  - self training
  - co-training
  - graph-based models
- self training 
  - What is the process?
    - First train on labeled data
    - Make predictions on the untrained data
    - From those predictions, only extract those with high confidence and move them to the training data
    - Repeat the process until convergence or no high-confidence exmples left in the unlabeled set.
  - Disadvantage is that you can still generate incorrect predictions and the mistakes can be amplified in the subsequent trainings of the model.
- co-training
  - requires two feature representations associated with the dataset which serve as two different views of the data
  - The representations are dissimilar and conditionally independet, but they can provide complementary information about the data. 
  - We cannot do this for image classification because we have only visual representations of the data.
- graph-based models
  - labeled and unlabeled samples are represented as different nodes in a graph
  - the edges in this graph denote the similarity between nodes.
  - The assumption in this approach is that nodes with strong edges are likely to share the same label.
  - The algorithm to compute the labels is:
    - The unlabeled nodes can be labeled using random-walk over the graph. Based on the strength of the edges.
    - The walk ends when a labeled node is reached 
    - A probability that the random walker started at a particular unlabeled node given that it ended at a specific labeled node is computed. i.e. 2 poitns are similar if they have indistinguishabel starting points.


Reference: 
- https://www.kdnuggets.com/2019/11/tips-class-imbalance-missing-labels.html

In [ ]:
# k-fold cross-validation, 
# ensemble methods, 
# or importance sampling

## Test the model on the new dataset 

In [ ]:

# This will shuffle the images in the dataset 
# before they are returned to the data loader,
# which should help ensure that the validation and training sets
# are more balanced.
class ShuffledImageFolder(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.indices = torch.randperm(len(dataset))

    def __getitem__(self, index):
        return self.dataset[self.indices[index]]

    def __len__(self):
        return len(self.dataset)

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = 'data/task1/labeled'
image_dataset_unshuffled = datasets.ImageFolder(data_dir, preprocess)
image_dataset = ShuffledImageFolder(image_dataset_unshuffled)

# print(image_datasets)
# We don't want to create a single dataset because we want to have a dataset for evaluation also
# dataloader  = torch.utils.data.DataLoader(image_dataset, batch_size = 4, shuffle = True, num_workers = 4)

# split data into train and val

dataset_size = len(image_dataset)
print(dataset_size)
split = int(dataset_size*0.8)
train_size = split
print(train_size)
val_size = dataset_size - split
print(val_size)
assert val_size+train_size == dataset_size
class_names = image_dataset.dataset.classes

# Create a sampler for the training set
train_sampler = SubsetRandomSampler(range(split))

# Create a sampler for the valuation set
val_sampler = SubsetRandomSampler(range(split, dataset_size))

# Create DataLoaders for the training and valuation sets
train_dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=32, sampler=train_sampler)
val_dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=32, sampler=val_sampler, shuffle=False)
dataloaders = {
    "train":train_dataloader,
    "val":val_dataloader,
}
dataset_sizes = {
    "train":train_size,
    "val":val_size,
}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Setting hyperparameters

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)